In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the dataset
DATASET_PATH = '/content/drive/MyDrive/Howard-Cloud-X'

# Directories for training and testing
train_dir = os.path.join(DATASET_PATH, "/content/drive/MyDrive/Howard-Cloud-X/train")
test_dir = os.path.join(DATASET_PATH, "/content/drive/MyDrive/Howard-Cloud-X/test")

# Data Preprocessing
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Building the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')  # Number of classes
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the Model
EPOCHS = 20
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data
)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the Model
model.save("cloud_type_classifier.h5")
print("Model saved as cloud_type_classifier.h5")


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from google.colab import files
import gdown

# Direct Google Drive download link
MODEL_URL = 'https://drive.google.com/uc?export=download&id=1hFObisDdYCBrbRmvqeRV0tAfpQ3q2kig'

# Download the model from Google Drive (only once)
model_path = '/content/cloud_type_classifier.h5'
if not os.path.exists(model_path):  # Check if the model is already downloaded
    print("Downloading the model...")
    gdown.download(MODEL_URL, model_path, quiet=False)
else:
    print("Model already downloaded.")

# Load the model
model = tf.keras.models.load_model(model_path)

# Define image preprocessing function
IMG_HEIGHT, IMG_WIDTH = 128, 128
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image to [0, 1]
    return img_array

# Load the class indices (you can modify this according to your model's classes)
class_indices = {0: 'Cumulus', 1: 'Stratus', 2: 'Cirrus'}  # Modify based on your model's class indices

# Function to upload and process the image
def upload_and_process_image():
    uploaded = files.upload()  # This will prompt the user to upload a file
    if uploaded:
        # Get the uploaded file name
        file_path = next(iter(uploaded))
        print(f"Image uploaded: {file_path}")
        return file_path
    else:
        print("No file uploaded")
        return None

# Function to make predictions after the image is uploaded
def predict_image(file_path):
    if not file_path:
        print("No image to predict")
        return

    # Preprocess the uploaded image
    processed_image = preprocess_image(file_path)

    # Predict the cloud type
    predictions = model.predict(processed_image)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)

    # Output the result
    print(f"Prediction: {class_indices[predicted_class]}")
    if confidence > 0.75:
        print(f"High confidence: It is likely a {class_indices[predicted_class]}")
    elif confidence > 0.5:
        print(f"Moderate confidence: It might be a {class_indices[predicted_class]}")
    else:
        print(f"Low confidence: It is unlikely to be a {class_indices[predicted_class]}")

    # Add extra condition for high confidence in "Cirrus" cloud type
    if class_indices[predicted_class] == "Cirrus" and confidence > 0.75:
        print("High chance of typhoon")

# Main function to run indefinitely and ask for image uploads
if __name__ == "__main__":
    print("Model is ready. You can upload an image of a cloud for prediction.")

    # Initialize the loop
    while user_input<2:
        file_path = upload_and_process_image()  # Allow user to upload the image
        user_input+=1
        if file_path:
            predict_image(file_path)  # Process the image and make predictions

        # Ask user if they want to upload another image
        user_input = 0
        if user_input ==2:
            print("Exiting the program. Goodbye!")
            break


Model already downloaded.


Model is ready. You can upload an image of a cloud for prediction.


TypeError: '<' not supported between instances of 'str' and 'int'